In [0]:
#!pip install tensorflow_datasets

In [0]:
#from tensorflow.keras.callbacks import TensorBoard
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow_datasets as tfds
import sys
import h5py
import datetime
#tf.compat.v1.enable_eager_execution()

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
sys.path.append('/content/drive/My Drive/convAE_2_classifier')

In [0]:
from dataset import Dataset
from convAE import convAE

In [0]:
#tf.keras.backend.clear_session()

In [23]:
dataset = Dataset('cats_vs_dogs')
IMG_SIZE = 256
input_shape = (IMG_SIZE, IMG_SIZE, 3)
dataset.set_img_size(IMG_SIZE)
train, val, test = dataset.format_data()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete3YOAYQ/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = val.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [0]:
autoencoder = convAE(input_shape=input_shape)
autoencoder_filters = [64, 128, 256, 512, 1024]
autoencoder.create_autoencoder_with_stacked_conv(filters=autoencoder_filters)
weights_dir = "/content/drive/My Drive/convAE_2_classifier/weights/"
filename = "convAE_stacked256_64-128-256-512-1024__20200323-163623_Epoch0040-Loss0.0039_.hdf5"
autoencoder.autoencoder.load_weights(weights_dir + filename)

In [26]:
mobileNet2 = load_model("/content/drive/My Drive/convAE_2_classifier/weights/classifier/denseNet169_256_weights.h5")

In [0]:
autoencoder.encoder.trainable = False
autoencoder.decoder.trainable = True
mobileNet2.trainable = False

In [0]:
# optimizer = tf.keras.optimizers.RMSprop()
# loss=tf.keras.losses.BinaryCrossentropy()
# mobileNet2.compile(optimizer=optimizer, loss=loss)

In [29]:
autoencoder.encoder.summary()
autoencoder.decoder.summary()
mobileNet2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 128)     5

In [0]:
combinedModels = tf.keras.Sequential([autoencoder.autoencoder, mobileNet2])

In [31]:
combinedModels.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 256, 256, 3)       47030211  
_________________________________________________________________
sequential (Sequential)      (None, 1)                 12644545  
Total params: 59,674,756
Trainable params: 28,175,235
Non-trainable params: 31,499,521
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 256, 256, 3)       47030211  
_________________________________________________________________
sequential (Sequential)      (None, 1)                 12644545  
Total params: 59,674,756
Trainable params: 28,175,235
Non-trainable params: 31,499,521
________________________________________________________

In [0]:
#combinedModels.load_weights("./drive/My Drive/convAE_2_classifier/weights/combined_models.0199-0.0357.hdf5")

In [33]:
listToStr = '-'.join(map(str, autoencoder_filters))
model_name = "convAE_stacked-DenseNet"
run_num = '_5_'
file_name = model_name + run_num + str(IMG_SIZE)+ '_' + listToStr + '_'
print(file_name)

convAE_stacked-DenseNet_5_256_64-128-256-512-1024_


In [34]:
dateTime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="./drive/My Drive/convAE_2_classifier/logs/" + file_name + "_logs_" + dateTime
weights_path="./drive/My Drive/convAE_2_classifier/weights/" + file_name + "_"  + dateTime + "_Epoch{epoch:04d}-Loss{loss:.4f}_.hdf5"
csv_path = "./drive/My Drive/convAE_2_classifier/logs/" + file_name +  "_model_history_" + dateTime + ".csv"

save_weights_callback = tf.keras.callbacks.ModelCheckpoint(filepath=weights_path , monitor='loss', save_weights_only=True, save_freq='epoch', period=10)
tensorboard_callback  = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch', histogram_freq=0, write_graph=True, write_images=True)
csv_logger = tf.keras.callbacks.CSVLogger(csv_path, separator=',', append=True)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

callbacks = [save_weights_callback,
             tensorboard_callback,
             csv_logger,
             #early_stop_callback
             ]

In [0]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
      return optimizer.lr 
    return lr

In [0]:
base_learning_rate = 0.00001
optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)
#optimizer = tf.keras.optimizers.Adamax(learning_rate=base_learning_rate, beta_1=0.9, beta_2=0.999)
#optimizer = tf.keras.optimizers.Nadam(learning_rate=base_learning_rate, beta_1=0.9, beta_2=0.999)
#optimizer = tf.keras.optimizers.SGD(learning_rate=base_learning_rate, momentum=0.9, nesterov=True)
#optimizer = tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
lr_metric = get_lr_metric(optimizer)
combinedModels.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy',
                       #lr_metric
                       ])

In [37]:
loss, acc  = combinedModels.evaluate(validation_batches, steps = 4)

4/4 [==============================] - 1s 208ms/step - loss: 1.6676 - accuracy: 0.8281


In [38]:
print(file_name)

convAE_stacked-DenseNet_5_256_64-128-256-512-1024_


In [39]:
history = combinedModels.fit(train_batches,
                             epochs=50,
                             #initial_epoch=history.epoch[-1],
                             validation_data=validation_batches,
                             validation_steps=4,
                             shuffle=True,
                             callbacks=callbacks)

Epoch 1/50
582/582 [==============================] - 397s 683ms/step - loss: 0.6351 - accuracy: 0.8706 - val_loss: 0.5700 - val_accuracy: 0.8203
Epoch 2/50
582/582 [==============================] - 392s 674ms/step - loss: 0.3805 - accuracy: 0.8733 - val_loss: 0.4478 - val_accuracy: 0.8594
Epoch 3/50
582/582 [==============================] - 391s 672ms/step - loss: 0.2790 - accuracy: 0.8965 - val_loss: 0.5013 - val_accuracy: 0.8672
Epoch 4/50
582/582 [==============================] - 392s 674ms/step - loss: 0.2224 - accuracy: 0.9155 - val_loss: 0.6128 - val_accuracy: 0.8281
Epoch 5/50
582/582 [==============================] - 394s 677ms/step - loss: 0.1759 - accuracy: 0.9336 - val_loss: 0.4503 - val_accuracy: 0.8516
Epoch 6/50
582/582 [==============================] - 394s 677ms/step - loss: 0.1517 - accuracy: 0.9406 - val_loss: 0.6986 - val_accuracy: 0.8281
Epoch 7/50
582/582 [==============================] - 394s 677ms/step - loss: 0.1236 - accuracy: 0.9516 - val_loss: 0.5628 -

In [0]:
# import os
# import shutil
# os.remove(log_dir)
# shutil.rmtree(log_dir)